In [ ]:
import scirpy as ir

In [ ]:
import scanpy as sc

In [ ]:
from glob import glob
import warnings
import tarfile

import anndata
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc

In [ ]:
import muon as mu

In [ ]:
import numpy as np

In [ ]:

samples = {
    "10mix1": {"group": "10mix"},
    "10mix2": {"group": "10mix"},
    "11mix1": {"group": "11mix"},
    "11mix2": {"group": "11mix"},
    "GF1": {"group": "GF"},
    "GF2": {"group": "GF"},

}    
    
# Create a list of AnnData objects (one for each sample)
adatas_tcr = {}
adatas_gex = {}
for sample, sample_meta in samples.items():
    adata_gex = sc.read_10x_h5(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/count/sample_filtered_feature_bc_matrix.h5")
    adata_tcr = ir.io.read_10x_vdj(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/vdj_t/filtered_contig_annotations.csv")
    # concatenation only works with unique gene names
    adata_gex.var_names_make_unique()
    adatas_tcr[sample] = adata_tcr
    adatas_gex[sample] = adata_gex

In [ ]:
samples = {
    "10mix-ICI1": {"group": "10mix", "condition":"ICI"},
    "10mix-ICI2": {"group": "10mix","condition":"ICI"},
    "11mix-ICI1": {"group": "11mix","condition":"ICI"},
    "11mix-ICI2": {"group": "11mix","condition":"ICI"},
    "GF-ICI1": {"group": "GF","condition":"No ICI"},
    "GF-ICI2": {"group": "GF","condition":"No ICI"},
    "GF-ICI1-plus": {"group": "GF-plus","condition":"ICI"},
    "GF-ICI2-plus": {"group": "GF-plus","condition":"ICI"},


}    
    
# Create a list of AnnData objects (one for each sample)
adatas_tcr_2021 = {}
adatas_gex_2021 = {}
for sample, sample_meta in samples.items():
    adata_gex_2021 = sc.read_10x_h5(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/count/sample_filtered_feature_bc_matrix.h5")
    adata_tcr_2021 = ir.io.read_10x_vdj(f"/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/{sample}/outs/per_sample_outs/{sample}/vdj_t/filtered_contig_annotations.csv")
    # concatenation only works with unique gene names
    adata_gex_2021.var_names_make_unique()
    adatas_tcr_2021[sample] = adata_tcr_2021
    adatas_gex_2021[sample] = adata_gex_2021

In [ ]:
samples = {
    "10mix-ICI1": {"group": "10mix", "condition":"ICI"},
    "10mix-ICI2": {"group": "10mix","condition":"ICI"},
    "11mix-ICI1": {"group": "11mix","condition":"ICI"},
    "11mix-ICI2": {"group": "11mix","condition":"ICI"},
    "GF-ICI1": {"group": "GF","condition":"No ICI"},
    "GF-ICI2": {"group": "GF","condition":"No ICI"},
    "GF-ICI1-plus": {"group": "GF-plus","condition":"ICI"},
    "GF-ICI2-plus": {"group": "GF-plus","condition":"ICI"},
    "10mix1": {"group": "10mix", "condition":"naive"},
    "10mix2": {"group": "10mix", "condition":"naive"},
    "11mix1": {"group": "11mix", "condition":"naive"},
    "11mix2": {"group": "11mix", "condition":"naive"},
    "GF1": {"group": "GF", "condition":"naive"},
    "GF2": {"group": "GF", "condition":"naive"},
}    

In [ ]:
# Merge anndata objects
adata_gex = anndata.concat(adatas_gex, index_unique="_")
adata_tcr = anndata.concat(adatas_tcr, index_unique="_")

In [ ]:
# Merge anndata objects
adata_gex_2021 = anndata.concat(adatas_gex_2021, index_unique="_")
adata_tcr_2021 = anndata.concat(adatas_tcr_2021, index_unique="_")

In [ ]:
adata_gex_merge = anndata.concat([adata_gex,adata_gex_2021], index_unique="_")
adata_tcr_merge = anndata.concat([adata_tcr,adata_tcr_2021], index_unique="_")

In [ ]:
# Set global met'adata on `mdata.obs`
adata_gex_merge.obs["sample"] = adata_gex_merge.obs_names.to_series().str.split("_", expand=True)[1]
adata_gex_merge.obs["group"] = adata_gex_merge.obs["sample"].map(lambda x: samples[x]["group"])
adata_gex_merge.obs["condition"] = adata_gex_merge.obs["sample"].map(lambda x: samples[x]["condition"])

In [ ]:
mdata = mu.MuData({"gex": adata_gex_merge, "airr": adata_tcr_merge})

In [ ]:
mdata

In [ ]:
mdata_2019 = mu.MuData({"gex": adata_gex, "airr": adata_tcr})

In [ ]:
mdata_2021 = mu.MuData({"gex": adata_gex_2021, "airr": adata_tcr_2021})

In [ ]:
mdata.obs["sample"] = mdata.obs_names.to_series().str.split("_", expand=True)[1]

In [ ]:
# Set global met'adata on `mdata.obs`
mdata.obs["sample"] = mdata.obs_names.to_series().str.split("_", expand=True)[1]
mdata.obs["group"] = mdata.obs["sample"].map(lambda x: samples[x]["group"])
mdata.obs["condition"] = mdata.obs["sample"].map(lambda x: samples[x]["condition"])

In [ ]:
sc.pp.filter_genes(mdata["gex"], min_cells=10)
sc.pp.filter_cells(mdata["gex"], min_genes=100)

In [ ]:
sc.pp.normalize_per_cell(mdata["gex"])
sc.pp.log1p(mdata["gex"])
sc.pp.highly_variable_genes(mdata["gex"], flavor="cell_ranger", n_top_genes=5000)
sc.tl.pca(mdata["gex"])
sc.pp.neighbors(mdata["gex"])

In [ ]:
sc.tl.umap(mdata["gex"])

In [ ]:
sc.tl.leiden(mdata["gex"])

In [ ]:
mdata["gex"].var

In [ ]:
mdata["gex"].obs["sample"] = mdata["gex"].obs_names.to_series().str.split("_", expand=True)[1]
mdata["gex"].obs["group"] = mdata["gex"].obs["sample"].map(lambda x: samples[x]["group"])
mdata["gex"].obs["condition"] = mdata["gex"].obs["sample"].map(lambda x: samples[x]["condition"])

In [ ]:
def update_columns(row):
   
    if "ICI" not in row['sample']:
        row['origin'] = "colon"
    else:
         row['origin'] = "til"
        
    return row

# Apply the function to each row in the DataFrame
mdata["gex"].obs= mdata["gex"].obs.apply(update_columns, axis=1)

In [ ]:
mdata.update()

In [ ]:
mu.pp.intersect_obs(mdata)

In [ ]:
mdata

In [ ]:
sc.pl.umap(mdata["gex"], color = "leiden")

In [ ]:
mu.pl.embedding(
    mdata,
    basis="gex:umap",
    color=["gex:leiden","gex:group"],
    ncols=3,
    wspace=0.7,
)


In [ ]:
mu.pl.embedding(
    mdata,
    basis="gex:umap",
    color=["gex:origin","gex:group"],
    ncols=3,
    wspace=0.7,
)


In [ ]:
#### cell annotation

In [ ]:
mu.pl.embedding(
    mdata,
    basis="gex:umap",
    color=["Klrk1","Ifng","Klrc1","Ccl4"],
    ncols=3,
    wspace=0.7,
    vmax= "p99",
    cmap ="inferno"
)


In [ ]:
mu.pl.embedding(
    mdata,
    basis="gex:umap",
    color=["Gzma","Dapk2","Gzmk","Gpr55"],
    ncols=3,
    wspace=0.7,
    vmax= "p99",
    cmap ="inferno"
)


In [ ]:
mu.pl.embedding(
    mdata,
    basis="gex:umap",
    color=["Cd8a","Cd8b1","Ifng","Gzmb"],
    ncols=3,
    wspace=0.7,
    vmax= "p99",
    cmap ="inferno"
)


In [ ]:
mu.pl.embedding(
    mdata,
    basis="gex:umap",
    color=["gex:leiden"],
    ncols=3,
    wspace=0.7,
)

In [ ]:
# mitochondrial genes
mdata["gex"].var["mt"] = mdata.var_names.str.startswith("mt-")


In [ ]:
sc.pp.calculate_qc_metrics(
    mdata["gex"], qc_vars=["mt"], inplace=True, percent_top=[20], log1p=True
)

In [ ]:
mdata

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes=axes.flatten()
sc.pl.umap(
    mdata["gex"],
    color="pct_counts_mt",
    cmap="inferno",
    vmin=0,
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[0],
)
sc.pl.umap(
    mdata["gex"],
    color="total_counts",
    cmap="inferno",
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[1],
)
sc.pl.umap(
    mdata["gex"],
    color="n_genes_by_counts",
    cmap="inferno",
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[2],
)
sc.pl.umap(
    mdata["gex"],
    color="n_genes",
    cmap="inferno",
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[3],
)

In [ ]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [ ]:

from scipy.stats import median_abs_deviation

In [ ]:
mdata["gex"].obs["outlier"] = (
    is_outlier(mdata["gex"], "log1p_total_counts", 1)
    | is_outlier(mdata["gex"], "log1p_n_genes_by_counts", 1)
    | is_outlier(mdata["gex"], "pct_counts_in_top_20_genes",1)
)
mdata["gex"].obs.outlier.value_counts()

In [ ]:
mdata_qc = mdata["gex"][(~mdata["gex"].obs.outlier)]

In [ ]:
mdata_qc

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes=axes.flatten()
sc.pl.umap(
    mdata_qc,
    color="pct_counts_mt",
    cmap="inferno",
    vmin=0,
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[0],
)
sc.pl.umap(
    mdata_qc,
    color="total_counts",
    cmap="inferno",
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[1],
)
sc.pl.umap(
    mdata_qc,
    color="n_genes_by_counts",
    cmap="inferno",
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[2],
)
sc.pl.umap(
    mdata_qc,
    color="n_genes",
    cmap="inferno",
    vmax="p99",
    sort_order=False,
    show=False,
    ax=axes[3],
)

In [ ]:
#naive
mu.pl.embedding(
    mdata_qc,
    basis="X_umap",
    color=["Sell","Lef1"],
    ncols=3,
    wspace=0.7,
    vmax= "p99",
    cmap ="inferno"
)

In [ ]:
#ex
mu.pl.embedding(
    mdata_qc,
    basis="X_umap",
    color=[ "Cxcr6","Gzmb","Havcr2","Ctla4"],
    ncols=3,
    wspace=0.7,
    vmax= "p99",
    cmap ="inferno"
)

In [ ]:
#9,13, 00,2,12,3,4,21,1,18,17 tex 
#11,8,5,16,10,6,7 tn

cl_annotation = { 
    "0": "Naive",
    "1": "Resident Memory",
    "3":"Naive",
    "4":"Exhausted",
    "5":"Exhausted",
    "7":"Exhausted",
    "8":"Naive",
    "10":"IFg",
    "11":"Exhausted",

}

In [ ]:
mdata["gex"].obs["cell_type_coarse"] = mdata["gex"].obs.leiden.map(cl_annotation)

In [ ]:
mdata_2019["gex"].obs["cell_type_coarse"] = mdata_2019["gex"].obs.leiden.map(cl_annotation)

In [ ]:
mdata_2021["gex"].obs["cell_type_coarse"] = mdata_2021["gex"].obs.leiden.map(cl_annotation)

In [ ]:
mdata_2021.update()

In [ ]:
mdata_2019.update()

In [ ]:
mdata_qc.obs["cell_type_coarse"] = mdata_qc.obs.leiden.map(cl_annotation)

In [ ]:
naive - 0, 3,8
rm - 10,1
ex - 5,11,7,4
ifg - 

In [ ]:
mdata_qc = mdata_qc[mdata_qc.obs["cell_type_coarse"].isin(['Exhausted', 'Naive', 'Resident Memory'])]
                     

In [ ]:
mdata_qc.write_h5ad("mdata_qc.h5ad")

In [ ]:
set(mdata_qc.obs.cell_type_coarse)

In [ ]:
mu.pl.embedding(
      mdata_qc,
    basis="X_umap",
    color=["cell_type_coarse","origin"],
    ncols=3,
    wspace=0.7,
)

In [ ]:
mu.pl.embedding(
      mdata_qc,
    basis="X_umap",
    color=["cell_type_coarse"],
    ncols=3,
    wspace=0.7,
)

In [ ]:
#rm 


#ex
mu.pl.embedding(
    mdata_qc,
    basis="X_umap",
    color=["Jaml","Prf1","Gzmb"],
    ncols=3,
    wspace=0.7,
    vmax= "p99",
    cmap ="inferno"
)

In [ ]:
#ifg
mu.pl.embedding(
    mdata_qc,
    basis="X_umap",
    color=[ "Isg15","Ifit3","Ifit1","Stat1" ],
    ncols=3,
    wspace=0.7,
    vmax= "p99",
    cmap ="inferno"
)

In [ ]:
mdata.update()

In [ ]:
Tnaive - Ltb","Sell","Lef1","Nosip -- 3,8
Tex - Cxcr6","Gzmb","Havcr2","Ctla4"
Tsen - Plcg2","Igkc","Klrk1","Slamf7"
Tifn "Isg15","Ifit3","Ifit1","Stat1" 

In [ ]:
#naive
mu.pl.embedding(
    mdata_2021,
    basis="gex:umap",
    color=["Sell","Lef1"],
    ncols=3,
    wspace=0.7,
    vmax= "p99",
    cmap ="inferno"
)

In [ ]:
#rm
mu.pl.embedding(
    mdata_2021,
    basis="gex:umap",
    color=["Jaml","Prf1","Gzmb"],
    ncols=3,
    wspace=0.7,
    vmax= "p99",
    cmap ="inferno"
)

In [ ]:
#ex
mu.pl.embedding(
    mdata_2021,
    basis="gex:umap",
    color=[ "Cxcr6","Gzmb","Havcr2","Ctla4"],
    ncols=3,
    wspace=0.7,
    vmax= "p99",
    cmap ="inferno"
)

In [ ]:
#ifg
mu.pl.embedding(
    mdata_2021,
    basis="gex:umap",
    color=[ "Isg15","Ifit3","Ifit1","Stat1" ],
    ncols=3,
    wspace=0.7,
    vmax= "p99",
    cmap ="inferno"
)

In [ ]:
#ifg
mu.pl.embedding(
    mdata_2019,
    basis="gex:umap",
    color=[ "Isg15","Stat1","sample"],
    ncols=3,
    wspace=0.7,
    vmax= "p99",
    cmap ="inferno"
)

In [ ]:
mdata

In [ ]:
ir.pp.index_chains(mdata)

In [ ]:
mdata

In [ ]:
ir.tl.chain_qc(mdata)

In [ ]:
set(mdata.obs["gex:sample"])

In [ ]:
mdata_2019  = mdata[mdata.obs["gex:sample"].isin(['10mix1',
 '10mix2',
 '11mix1',
 '11mix2',
  'GF1',
 'GF2'])]

In [ ]:
mdata_2021  = mdata[mdata.obs["gex:sample"].isin(['10mix-ICI1',
 '10mix-ICI2',
 '11mix-ICI1',
 '11mix-ICI2',
 'GF-ICI1',
 'GF-ICI1-plus',
 'GF-ICI2',
 'GF-ICI2-plus'])]

In [ ]:
mdata_2019

In [ ]:
mdata_2021

In [ ]:
_ = ir.pl.group_abundance(mdata_2019, groupby="airr:receptor_subtype", target_col="gex:sample", )

In [ ]:
_ = ir.pl.group_abundance(mdata_2021, groupby="airr:receptor_subtype", target_col="gex:sample", )

In [ ]:
mdata.update()

In [ ]:
_ = ir.pl.group_abundance(mdata, groupby="airr:receptor_subtype", target_col="gex:cell_type_coarse", )

In [ ]:
_ = ir.pl.group_abundance(mdata_2019, groupby="airr:receptor_subtype", target_col="gex:cell_type_coarse", )

In [ ]:
_ = ir.pl.group_abundance(mdata_2021, groupby="airr:receptor_subtype", target_col="gex:cell_type_coarse", )

In [ ]:
mdata_qc.obs["cell_type_coarse"] = mdata_qc.obs.leiden.map(cl_annotation)

In [ ]:
_ = ir.pl.group_abundance(mdata, groupby="airr:receptor_subtype", target_col="gex:origin")

In [ ]:
_ = ir.pl.group_abundance(mdata, groupby="airr:chain_pairing", target_col="gex:origin")

In [ ]:
_ = ir.pl.group_abundance(mdata, groupby="airr:chain_pairing", target_col="gex:cell_type_coarse")

In [ ]:
_ = ir.pl.group_abundance(mdata_2019, groupby="airr:chain_pairing", target_col="gex:cell_type_coarse")

In [ ]:
_ = ir.pl.group_abundance(mdata, groupby="gex:cell_type_coarse", target_col="airr:chain_pairing")

In [ ]:
_ = ir.pl.group_abundance(mdata_2019, groupby="gex:cell_type_coarse", target_col="airr:chain_pairing")

In [ ]:
_ = ir.pl.group_abundance(mdata_2021, groupby="gex:cell_type_coarse", target_col="airr:chain_pairing")

In [ ]:
_ = ir.pl.group_abundance(mdata_2021, groupby="airr:chain_pairing", target_col="gex:cell_type_coarse")

In [ ]:
print(
    "Fraction of cells with more than one pair of TCRs: {:.2f}".format(
        np.sum(mdata.obs["airr:chain_pairing"].isin(["extra VJ", "extra VDJ", "two full chains", "multichain"]))
        / mdata["airr"].n_obs
    )
)

In [ ]:
mu.pl.embedding(mdata, basis="gex:umap", color="airr:chain_pairing", groups="two full chains")

In [ ]:
#mu.pp.filter_obs(mdata, "airr:chain_pairing", lambda x: x != "two full chains")

In [ ]:
import numpy as np

In [ ]:
mu.pp.filter_obs(mdata, "airr:chain_pairing", lambda x: ~np.isin(x, ["orphan VDJ", "orphan VJ"]))

In [ ]:
mu.pp.filter_obs(mdata_2019, "airr:chain_pairing", lambda x: ~np.isin(x, ["orphan VDJ", "orphan VJ"]))

In [ ]:
mu.pp.filter_obs(mdata_2021, "airr:chain_pairing", lambda x: ~np.isin(x, ["orphan VDJ", "orphan VJ"]))

In [ ]:
mdata["gex"].var

In [ ]:
ax = ir.pl.group_abundance(mdata, groupby="airr:chain_pairing", target_col="gex:origin")

In [ ]:
ax = ir.pl.group_abundance(mdata, groupby="airr:chain_pairing", target_col="gex:origin")

In [ ]:
ax = ir.pl.group_abundance(mdata, groupby="gex:sample", target_col="airr:chain_pairing")

In [ ]:
ax = ir.pl.group_abundance(mdata_2019, groupby="gex:sample", target_col="airr:chain_pairing")

In [ ]:
ax = ir.pl.group_abundance(mdata_2021, groupby="gex:sample", target_col="airr:chain_pairing")

In [ ]:
mdata

In [ ]:
mdata_2019

In [ ]:
mu.pp.intersect_obs(mdata)

In [ ]:
# using default parameters, `ir_dist` will compute nucleotide sequence identity
ir.pp.ir_dist(mdata_2019 )

In [ ]:
ir.tl.define_clonotypes(mdata_2019, receptor_arms="all",dual_ir="primary_only")

In [ ]:
mdata["airr"].obs

In [ ]:
mdata["airr"].var

In [ ]:
mdata["gex"]

In [ ]:
sc.pp.neighbors(mdata["airr"])

In [ ]:
muon.tl.leiden(mdata)
muon.tl.umap(mdata)

In [ ]:
mdata["airr"]

In [ ]:
mdata["gex"]

In [ ]:
sc.pl.umap(mdata["gex"], color="Cxcr3")

In [ ]:
sc.pl.umap(mdata["gex"], color="leiden")

In [ ]:
sc.pl.umap(mdata_2019["gex"], color=["Cxcr3","sample"])

In [ ]:
sc.pl.umap(mdata_2021["gex"], color="Cxcr3")

In [ ]:
mdata_2021

In [ ]:
sc.pl.umap(mdata_2019["gex"], color="sample")

### Compositional analysis

In [ ]:
sample

In [ ]:
adata_tumor = adata_gex_merge[(adata_gex_merge.obs["origin"] == "tumor_primary")] #tumor 

In [ ]:
adata_normal = subset_adata[(subset_adata.obs["origin"] == "normal_adjacent")] #normal
adata_normal = adata_normal[(adata_normal.obs["cell_type_major"] != "Tumor cells")] #normal

In [ ]:
set(frac_by_condition.origin)

In [ ]:
frac_by_condition = (
    adata_gex_merge.obs.groupby(["group", "condition"])
    .apply(lambda x: x.value_counts("sample", normalize=False))
    .reset_index(name="n_cells")
    .assign(condition=lambda x: x["group"].astype(str))
)


In [ ]:
frac_pivot = (
    frac_by_condition.pivot(
        index=["group", "condition"],
        columns="sample",
        values="n_cells",
    )
    .reset_index()
    #.drop(columns="other")
)


In [ ]:
data_all = scc_dat.from_pandas(
    frac_pivot, covariate_columns=["donor_id", "dataset", "sex", "tumor_stage","condition","origin"]
)

In [ ]:
data_all.obs["sex"] = pd.Categorical(
    data_all.obs["sex"], categories=["male", "female"]
)

In [ ]:
data_all._sanitize()

In [ ]:
scc_viz.boxplots(data_all, feature_name="sex", figsize=(12, 5), cmap = "inferno")

#plt.savefig( "figures/cell_proportion_tumor_and_normal_sex.png")


In [ ]:
scc_viz.boxplots(data_all, feature_name="origin", figsize=(12, 5), cmap = "inferno")

#plt.savefig( "figures/cell_proportion_tumor_and_normal_origin.png")


In [ ]:
scc_viz.stacked_barplot(data_all, feature_name="sex", figsize = (3,6))
#plt.savefig( "figures/cell_proportion_tumor_and_normal_sex_barplot.png")

In [ ]:
scc_viz.stacked_barplot(data_all, feature_name="origin", figsize = (3,6))
#plt.savefig( "figures/cell_proportion_tumor_and_normal_origin_barplot.png")

In [ ]:
#############################################################

In [ ]:
# Load in the H5AD data
path = "/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/results_by_sample/merge_and_solo_samples"
adata = sc.read_h5ad(
    f"{path}/adata_nodoublet.h5ad"
)


In [ ]:
adata_vdj_10mix1_2019 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/10mix1/outs/multi/vdj_t/all_contig_annotations.csv")
adata_vdj_10mix2_2019 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/10mix2/outs/multi/vdj_t/all_contig_annotations.csv")
adata_vdj_11mix1_2019 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/11mix1/outs/multi/vdj_t/all_contig_annotations.csv")
adata_vdj_11mix2_2019 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/11mix2/outs/multi/vdj_t/all_contig_annotations.csv")
adata_vdj_GF1_2019 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/GF1/outs/multi/vdj_t/all_contig_annotations.csv")

############ 2021
adata_vdj_10mix_ICI1_2021 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/10mix-ICI1/outs/multi/vdj_t/all_contig_annotations.csv")
adata_vdj_10mix_ICI2_2021 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/10mix-ICI2/outs/multi/vdj_t/all_contig_annotations.csv")
adata_vdj_11mix_ICI1_2021 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/11mix-ICI1/outs/multi/vdj_t/all_contig_annotations.csv")
adata_vdj_11mix_ICI2_2021 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/11mix-ICI2/outs/multi/vdj_t/all_contig_annotations.csv")
adata_vdj_GF_ICI1_2021 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/GF-ICI1/outs/multi/vdj_t/all_contig_annotations.csv")
adata_vdj_GF_ICI2_2021 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/GF-ICI2/outs/multi/vdj_t/all_contig_annotations.csv")
adata_vdj_GF_ICI1_plus_2021 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/GF-ICI1-plus/outs/multi/vdj_t/all_contig_annotations.csv")
adata_vdj_GF_ICI2_plus_2021 = ir.io.read_10x_vdj("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/GF-ICI2-plus/outs/multi/vdj_t/all_contig_annotations.csv")

In [ ]:
adata_gex_10mix1_2019.obs["sample_id"]="10mix1_2019"
adata_gex_10mix2_2019.obs["sample_id"]="10mix2_2019"
adata_gex_11mix1_2019.obs["sample_id"]="11mix1_2019"
adata_gex_11mix2_2019.obs["sample_id"]="11mix2_2019"
adata_gex_GF1_2019.obs["sample_id"]="GF1_2019"


###############################################
adata_gex_10mix_ICI1_2021.obs["sample_id"]="10mix-ICI1_2021"
adata_gex_10mix_ICI2_2021.obs["sample_id"]="10mix-ICI2_2021"
adata_gex_11mix_ICI1_2021.obs["sample_id"]="11mix-ICI1_2021"
adata_gex_11mix_ICI2_2021.obs["sample_id"]="11mix-ICI2_2021"
adata_gex_GF_ICI1_2021.obs["sample_id"]="GF-ICI1_2021"
adata_gex_GF_ICI2_2021.obs["sample_id"]="GF-ICI2_2021"
adata_gex_GF_ICI1_plus_2021.obs["sample_id"]="GF-ICI1_plus_2021"
adata_gex_GF_ICI2_plus_2021.obs["sample_id"]="GF-ICI2_plus_2021"

In [ ]:
adata_gex_10mix1_2019 =sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/10mix1/outs/per_sample_outs/10mix1/count/sample_filtered_feature_bc_matrix")
adata_gex_10mix2_2019 = sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/10mix2/outs/per_sample_outs/10mix2/count/sample_filtered_feature_bc_matrix")
adata_gex_11mix1_2019 =sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/11mix1/outs/per_sample_outs/11mix1/count/sample_filtered_feature_bc_matrix")
adata_gex_11mix2_2019 = sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/11mix2/outs/per_sample_outs/11mix2/count/sample_filtered_feature_bc_matrix")
adata_gex_GF1_2019 =sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/GF1/outs/per_sample_outs/GF1/count/sample_filtered_feature_bc_matrix")

############ 2021
adata_gex_10mix_ICI1_2021 = sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/10mix-ICI1/outs/per_sample_outs/10mix-ICI1/count/sample_filtered_feature_bc_matrix")
adata_gex_10mix_ICI2_2021 =sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/10mix-ICI2/outs/per_sample_outs/10mix-ICI2/count/sample_filtered_feature_bc_matrix")
adata_gex_11mix_ICI1_2021 =sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/11mix-ICI1/outs/per_sample_outs/11mix-ICI1/count/sample_filtered_feature_bc_matrix")
adata_gex_11mix_ICI2_2021 = sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/11mix-ICI2/outs/per_sample_outs/11mix-ICI2/count/sample_filtered_feature_bc_matrix")
adata_gex_GF_ICI1_2021 = sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/GF-ICI1/outs/per_sample_outs/GF-ICI1/count/sample_filtered_feature_bc_matrix")
adata_gex_GF_ICI2_2021 = sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/GF-ICI2/outs/per_sample_outs/GF-ICI2/count/sample_filtered_feature_bc_matrix")
adata_gex_GF_ICI1_plus_2021 = sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/GF-ICI1-plus/outs/per_sample_outs/GF-ICI1-plus/count/sample_filtered_feature_bc_matrix")
adata_gex_GF_ICI2_plus_2021 = sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/GF-ICI2-plus/outs/per_sample_outs/GF-ICI2-plus/count/sample_filtered_feature_bc_matrix")

In [ ]:
adata_vdj_list = [adata_vdj_10mix2_2019,adata_vdj_11mix1_2019,adata_vdj_11mix2_2019,adata_vdj_GF1_2019,adata_vdj_10mix_ICI1_2021,adata_vdj_10mix_ICI2_2021,adata_vdj_11mix_ICI1_2021,adata_vdj_11mix_ICI2_2021,adata_vdj_GF_ICI1_2021,adata_vdj_GF_ICI2_2021,adata_vdj_GF_ICI1_plus_2021,adata_vdj_GF_ICI2_plus_2021]

In [ ]:
adata_vdj = adata_vdj_10mix1_2019.concatenate(adata_vdj_list)

In [ ]:
adata_gex_list = [adata_gex_10mix2_2019,adata_gex_11mix1_2019,adata_gex_11mix2_2019,adata_gex_GF1_2019,adata_gex_10mix_ICI1_2021,adata_gex_10mix_ICI2_2021,adata_gex_11mix_ICI1_2021,adata_gex_11mix_ICI2_2021,adata_gex_GF_ICI1_2021,adata_gex_GF_ICI2_2021,adata_gex_GF_ICI1_plus_2021,adata_gex_GF_ICI2_plus_2021]

In [ ]:
adata_gex = adata_gex_10mix1_2019.concatenate(adata_gex_list)

In [ ]:
adata_vdj.write_h5ad("adata_vdj.h5ad")

In [ ]:
adata_gex.write_h5ad("adata_gex.h5ad")

In [ ]:
adata_vdj

In [ ]:
adata_vdj_10mix1_2019

In [ ]:
adata_gex_10mix1_2019  = sc.read_10x_mtx("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2019-10-29_sorted_cd8/analyses_icbi/10mix1/outs/per_sample_outs/10mix1/count/sample_filtered_feature_bc_matrix")

In [ ]:
adata_vdj_10mix1_2019

In [ ]:
adata_vdj_10mix1_2019.obs

In [ ]:
adata_vdj_10mix1_2019.var

In [ ]:
adata_gex_10mix1_2019

In [ ]:
adata_gex_10mix1_2019.obs

In [ ]:
adata_gex_10mix1_2019.var

In [ ]:
pd.DataFrame([x for x in adata_gex_10mix1_2019.var.index[~adata_gex_10mix1_2019.var.gene_ids.str.contains("ENSMUSG")]], columns=['Antibody'])

In [ ]:
mudata_10mix1_2019 = mu.MuData({"gex": adata_gex_10mix1_2019, "airr": adata_vdj_10mix1_2019})

In [ ]:
mudata_10mix1_2019

In [ ]:
mu.pp.intersect_obs(mudata_10mix1_2019)

In [ ]:
mdata = mudata_10mix1_2019

In [ ]:
sc.pp.filter_genes(mdata["gex"], min_cells=10)
sc.pp.filter_cells(mdata["gex"], min_genes=100)
sc.pp.normalize_total(mdata["gex"])
sc.pp.log1p(mdata["gex"])

# Simplified preprocessing steps:
sc.pp.highly_variable_genes(mdata["gex"], n_top_genes=5000)
sc.tl.pca(mdata["gex"])
sc.pp.neighbors(mdata["gex"])

In [ ]:
mdata["gex"].obs["sample_id"] = "10mix_1_2019"

In [ ]:
mdata.update()

In [ ]:
ir.pp.index_chains(mdata)

In [ ]:
ir.tl.chain_qc(mdata)

_ = ir.pl.group_abundance(
   mdata, groupby="airr:receptor_type", target_col="gex:sample_id"
)

In [ ]:
_ = ir.pl.group_abundance(
    mdata, groupby="airr:receptor_subtype", target_col="gex:sample_id"
)

In [ ]:
mu.pp.filter_obs(mdata, "airr:chain_pairing", lambda x: x != "multichain")

In [ ]:
_ = ir.pl.group_abundance(
    mdata, groupby="airr:chain_pairing", target_col="gex:sample_id"
)

In [ ]:
mu.pp.filter_obs(mdata, "airr:chain_pairing", lambda x: x == "single pair")

In [ ]:
sc.tl.umap(mdata["gex"])
sc.tl.leiden(mdata["gex"])

In [ ]:
mudata_10mix1_2019

In [ ]:
sc.pl.umap(mdata["gex"], color=["leiden"], wspace=0.3)

In [ ]:
ir.pp.ir_dist(mdata, metric='identity', sequence='nt')

In [ ]:
ir.tl.define_clonotypes(mdata, receptor_arms="all", dual_ir="primary_only")

In [ ]:
print(pd.__version__)

In [ ]:
print(sc.__version__)